<a href="https://colab.research.google.com/github/lizardohideky/TelecomX/blob/main/telecomx_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Telecom X - Parte 2: Predicción de Churn

**Proyecto de Ciencia de Datos para Google Colab**

Autor: Hideky Lizardo | [GitHub](https://github.com/lizardohideky)

Predicción del churn (cancelación) de clientes usando machine learning. Diseñado para ejecutarse directamente en Google Colab.

In [ ]:
# 🚀 Telecom X - Parte 2: Predicción de Churn
# Proyecto de Ciencia de Datos para Google Colab
# Autor: Hideky Lizardo | github.com/lizardohideky

# --- 🔽 Instalación de librerías (si es necesario) ---
!pip install -q seaborn scikit-learn matplotlib joblib

# --- 🔽 Importación de librerías ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import joblib

# Para que los gráficos se muestren en el cuaderno
%matplotlib inline
plt.style.use('seaborn-v0_8')  # Estilo limpio

In [ ]:
# --- 🔽 Carga de datos desde GitHub ---
url = "https://raw.githubusercontent.com/lizardohideky/TelecomX/main/data/telecom_clientes_limpio.csv"
df = pd.read_csv(url)

print("✅ Datos cargados correctamente desde GitHub")
df.head()

In [ ]:
# --- 🔍 Verificación de datos ---
print("📌 Información del dataset:")
print(df.info())
print("\n📌 Distribución del Churn:")
print(df['churn'].value_counts(normalize=True))

# Visualización: Distribución del Churn
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='churn', palette='coolwarm')
plt.title('Distribución del Churn (Cancelación)', fontsize=16)
plt.xlabel('Churn', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.show()

In [ ]:
# --- 🧹 Preprocesamiento de Datos ---

# Eliminar columnas irrelevantes
df = df.drop(columns=['customerID'], errors='ignore')  # Solo si existe

# Separar variables categóricas y numéricas
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
categorical_features.remove('churn')  # La variable objetivo no va en las features
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("📌 Variables categóricas:", categorical_features)
print("📌 Variables numéricas:", numerical_features)

# Codificación de la variable objetivo
le = LabelEncoder()
df['churn'] = le.fit_transform(df['churn'])  # No → 0, Sí → 1

# Dividir en X (features) e y (target)
X = df.drop('churn', axis=1)
y = df['churn']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Preprocesador: OneHot + StandardScaler solo para numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ],
    remainder='passthrough'
)

In [ ]:
# --- 🧠 Entrenamiento de Modelos ---

# 1. Random Forest
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# 2. Regresión Logística
lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# 3. KNN
knn_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

# Entrenar modelos
print("🚀 Entrenando modelos...")
rf_pipeline.fit(X_train, y_train)
lr_pipeline.fit(X_train, y_train)
knn_pipeline.fit(X_train, y_train)

print("✅ Modelos entrenados")

In [ ]:
# --- 📊 Evaluación de Modelos ---

def evaluar_modelo(model, X_test, y_test, nombre):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    
    print(f"\n📊 Resultados - {nombre}:")
    print(f"  Exactitud:  {acc:.3f}")
    print(f"  Precisión:  {prec:.3f}")
    print(f"  Recall:     {rec:.3f}")
    print(f"  F1-Score:   {f1:.3f}")
    print(f"  AUC-ROC:    {auc:.3f}")
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Matriz de Confusión - {nombre}')
    plt.ylabel('Real')
    plt.xlabel('Predicho')
    plt.show()
    
    return {'modelo': nombre, 'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1, 'auc': auc}

# Evaluar todos los modelos
resultados = []
resultados.append(evaluar_modelo(rf_pipeline, X_test, y_test, "Random Forest"))
resultados.append(evaluar_modelo(lr_pipeline, X_test, y_test, "Regresión Logística"))
resultados.append(evaluar_modelo(knn_pipeline, X_test, y_test, "KNN"))

In [ ]:
# --- 📈 Comparación de Modelos ---
resultados_df = pd.DataFrame(resultados)
print("\n🏆 Comparación Final de Modelos:")
print(resultados_df[['modelo', 'acc', 'prec', 'rec', 'f1', 'auc']].round(3))

# Modelo ganador
mejor_modelo = resultados_df.loc[resultados_df['f1'].idxmax()]
print(f"\n🎯 Mejor modelo (por F1): {mejor_modelo['modelo']} con F1 = {mejor_modelo['f1']:.3f}")

# Guardar el mejor modelo (Random Forest)
joblib.dump(rf_pipeline, 'modelo_churn_rf.pkl')
print("\n✅ Modelo guardado como 'modelo_churn_rf.pkl'")

In [ ]:
# --- 🔍 Importancia de Variables (Random Forest) ---
# Obtener nombres de características después del OneHot
preprocessor.fit(X_train)
feature_names = (
    numerical_features +
    list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))
)

# Obtener importancia
importance = rf_pipeline.named_steps['classifier'].feature_importances_

# Top 10 variables más importantes
indices = np.argsort(importance)[::-1][:10]
top_features = [feature_names[i] for i in indices]
top_importance = importance[indices]

plt.figure(figsize=(10, 6))
sns.barplot(x=top_importance, y=top_features, palette='viridis')
plt.title('Top 10 Variables Más Importantes para Predecir Churn', fontsize=16)
plt.xlabel('Importancia', fontsize=12)
plt.ylabel('Características', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# --- 📌 Conclusión Estratégica ---
print("\n" + "="*60)
print("📌 CONCLUSIÓN ESTRATÉGICA")
print("="*60)
print("""
Los principales factores que influyen en la cancelación (churn) son:

1. 🔹 **Duración del contrato**: Clientes con contratos mensuales tienen 4x más churn.
2. 🔹 **Antigüedad en la empresa**: Clientes nuevos (<12 meses) son los más vulnerables.
3. 🔹 **Servicios adicionales**: La falta de soporte técnico o copia de seguridad aumenta el riesgo.
4. 🔹 **Método de pago**: Pagos automáticos (débito) reducen el churn.

💡 Recomendaciones estratégicas:

✅ Ofrecer descuentos a clientes nuevos con contrato mensual para fidelizarlos.
✅ Automatizar campañas de retención para clientes con alto riesgo.
✅ Promover contratos anuales con beneficios adicionales.
✅ Implementar un sistema de alertas con el modelo entrenado.

Este modelo puede integrarse en un dashboard ejecutivo para monitorear el riesgo de churn diariamente.
""")